In [ ]:
import pandas as pd
import re

# 2. Load the dataset
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/All_Trans_2000_2024.csv")

# 3. Inspect columns (optional, good practice)
print(df.columns)

# 4. Define unwanted transfer fee keywords
unwanted_fees = ['ablösefrei', 'Leihe', 'Leihgebühr', '?']

# Escape special characters in the unwanted_fees list for regex
escaped_unwanted_fees = [re.escape(fee) for fee in unwanted_fees]

# 5. Remove rows containing unwanted values in Transfer_Fee column
df_clean = df[~df['Transfer_Fee'].str.contains(
    '|'.join(escaped_unwanted_fees),
    case=False,
    na=False
)]

# 6. Reset index after filtering
df_clean = df_clean.reset_index(drop=True)

# 7. Check result
print("Original rows:", len(df))
print("Remaining rows after cleaning:", len(df_clean))

# 8. Save cleaned dataset
df_clean.to_csv("All_Trans_2000_2024_clean_step1.csv", index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Index(['Unnamed: 0', 'Player', 'Age', 'Club_Left', 'League_Left',
       'Club_Joined', 'League_Joined', 'Transfer_Fee', 'Transfer_Window',
       'Performance_Year'],
      dtype='object')
Original rows: 43304
Remaining rows after cleaning: 21486


In [ ]:
df_clean.head(10)

,Unnamed: 0,Player,Age,Club_Left,League_Left,Club_Joined,League_Joined,Transfer_Fee,Transfer_Window,Performance_Year
0,0,Luís Figo,27,FC Barcelona,LaLiga,Real Madrid,LaLiga,"60,00 Mio. €",2000,1999
1,1,Hernán Crespo,25,AC Parma,Serie A,Lazio Rom,Serie A,"56,81 Mio. €",2000,1999
2,2,Gabriel Batistuta,31,AC Florenz,Serie A,AS Rom,Serie A,"36,15 Mio. €",2000,1999
3,3,Nicolas Anelka,21,Real Madrid,LaLiga,FC Paris Saint-Germain,Division 1,"34,50 Mio. €",2000,1999
4,4,Rio Ferdinand,22,West Ham United,Premier League,Leeds United,Premier League,"26,00 Mio. €",2000,1999
5,5,Savo Milošević,26,Real Saragossa,LaLiga,AC Parma,Serie A,"25,00 Mio. €",2000,1999
6,6,Flávio Conceição,26,Deportivo La Coruna,LaLiga,Real Madrid,LaLiga,"25,00 Mio. €",2000,1999
7,7,David Trezeguet,22,AS Monaco,Division 1,Juventus Turin,Serie A,"23,24 Mio. €",2000,1999
8,8,Matías Almeyda,26,Lazio Rom,Serie A,AC Parma,Serie A,"23,00 Mio. €",2000,1999
9,9,Claudio López,25,FC Valencia,LaLiga,Lazio Rom,Serie A,"23,00 Mio. €",2000,1999


In [ ]:
# 9. Function to convert transfer fee string to million euros
def convert_fee_to_million(fee):
    if pd.isna(fee):
        return None

    # Normalize string
    fee = fee.replace(',', '.').strip()

    # Million euros
    if 'Mio' in fee:
        value = fee.replace('Mio. €', '').replace('Mio €', '').strip()
        return float(value)

    # Thousand euros
    if 'Tsd' in fee:
        value = fee.replace('Tsd. €', '').replace('Tsd €', '').strip()
        return float(value) / 1000

    # If unexpected format
    return None


# 10. Apply conversion
df_clean['Transfer_Fee_In_MillionEuro'] = df_clean['Transfer_Fee'].apply(convert_fee_to_million)

# 11. Drop rows that could not be converted (safety step)
df_clean = df_clean.dropna(subset=['Transfer_Fee_In_MillionEuro'])

# 12. Reset index
df_clean = df_clean.reset_index(drop=True)

# 13. Check results
print(df_clean[['Transfer_Fee', 'Transfer_Fee_In_MillionEuro']].head(10))
print("Rows after numeric conversion:", len(df_clean))


   Transfer_Fee  Transfer_Fee_In_MillionEuro
0  60,00 Mio. €                        60.00
1  56,81 Mio. €                        56.81
2  36,15 Mio. €                        36.15
3  34,50 Mio. €                        34.50
4  26,00 Mio. €                        26.00
5  25,00 Mio. €                        25.00
6  25,00 Mio. €                        25.00
7  23,24 Mio. €                        23.24
8  23,00 Mio. €                        23.00
9  23,00 Mio. €                        23.00
Rows after numeric conversion: 21446


In [ ]:
df_clean.head(5)

,Unnamed: 0,Player,Age,Club_Left,League_Left,Club_Joined,League_Joined,Transfer_Fee,Transfer_Window,Performance_Year,Transfer_Fee_In_MillionEuro
0,0,Luís Figo,27,FC Barcelona,LaLiga,Real Madrid,LaLiga,"60,00 Mio. €",2000,1999,60.00
1,1,Hernán Crespo,25,AC Parma,Serie A,Lazio Rom,Serie A,"56,81 Mio. €",2000,1999,56.81
2,2,Gabriel Batistuta,31,AC Florenz,Serie A,AS Rom,Serie A,"36,15 Mio. €",2000,1999,36.15
3,3,Nicolas Anelka,21,Real Madrid,LaLiga,FC Paris Saint-Germain,Division 1,"34,50 Mio. €",2000,1999,34.50
4,4,Rio Ferdinand,22,West Ham United,Premier League,Leeds United,Premier League,"26,00 Mio. €",2000,1999,26.00


In [ ]:
# 14. Standardize league name: Division 1 -> Ligue 1
df_clean['League_Joined'] = df_clean['League_Joined'].replace(
    {'Division 1': 'Ligue 1'}
)

# Optional: check unique league names
print(sorted(df_clean['League_Joined'].unique()))


['1 . Liga Gr. 2 (- 11/12)', '1. HNL', '1. divisjon', '1.Division', '1.HNL', '1.Lig', '1B Pro League', '2. Bundesliga', '2. Liga inter - Gr. 3 (- 11/12)', '2ª B - Grupo I', '2ª B - Grupo II', '2ª B - Grupo III', '2ª División', '3. Liga', 'A Grupa', 'A Lyga', "A' Ethniki", "A' Katigoria", 'A-Junioren Eredivisie', 'Albanien', 'Allsvenskan', 'Angola', 'Argentinien', 'Ascenso Clausura', 'Ascenso MX Cl.', 'Aserbaidschan', 'Australien', 'Azadegan League (- 2001)', "B' Ethniki", 'Belgien', 'Beloften Eredivisie', 'Betclic 1 Liga', 'Botola Pro Inwi', 'Brasilien', 'Bulgarien', 'Bundesliga', 'CPL', 'Camp. Uruguayo Esp.', 'Castle PSL', 'Challenge League', 'Championship', 'Chile', 'China', 'Clausura', 'Copa de la Liga', 'Corgon Liga', 'DStv Premiership', 'Deutschland', 'Division 2', 'Divizia A', 'Divizia Nationala (-2017)', 'Dänemark', 'EXQI League', 'Ecuador', 'Eerste Divisie', 'Ekstraklasa', 'Eliteserien', 'England', 'Eredivisie', 'Erovnuli Liga', 'Erste Liga', 'Estland', 'FNL', 'Faxe Kondi Liga'

In [ ]:
# 15. Keep only rows where League_Joined is in Top 5 leagues
top_5_leagues = [
    'Premier League',
    'LaLiga',
    'Bundesliga',
    'Serie A',
    'Ligue 1'
]
df_top5 = df_clean[df_clean['League_Joined'].isin(top_5_leagues)]

# Reset index
df_top5 = df_top5.reset_index(drop=True)

# Check results
print("Rows before Top 5 filter:", len(df_clean))
print("Rows after Top 5 filter:", len(df_top5))

# Optional: count transfers per league
print(df_top5['League_Joined'].value_counts())


Rows before Top 5 filter: 21446
Rows after Top 5 filter: 9770
League_Joined
Premier League    2544
Serie A           2356
Bundesliga        1858
Ligue 1           1534
LaLiga            1478
Name: count, dtype: int64


In [ ]:
df_top5.to_csv(
    "/content/drive/MyDrive/All_Trans_2000_2024_Top5Leagues.csv",
    index=False
)